In [5]:
from pyspark.sql import SparkSession

# spark 세션을 만듦
# 이름을 지정하면 충돌이 안남
spark = SparkSession.builder.appName('PySpark').getOrCreate()

24/12/03 10:27:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [83]:
#spark 세션의 모드 확인하기
spark.sparkContext.master

'local[*]'

### RDD 객체 사용
spark의 기본 데이터 형식  
python에서 리스트 생성하듯이 하면 되지만, 분산 데이터 객체이기에, session에 등록해야함

In [6]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5]) #paralleize : 병렬임. 하나가 깨져도 다른 곳에서 작동하는 느낌
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [8]:
rdd.take(5)

[1, 2, 3, 4, 5]

In [20]:
squared_rdd = rdd.map( lambda x : x*x )
squared_rdd.take(5)

[1, 4, 9, 16, 25]

In [21]:
squared_rdd.collect() #collect는 sparkUI에 job에 뜸

[1, 4, 9, 16, 25]

### 데이터 프레임 사용

In [23]:
data = [('Alice', 1), ('Bob',2 ), ('Charlie',3)]
df = spark.createDataFrame(data, ['Name', 'Value'])
df

DataFrame[Name: string, Value: bigint]

In [24]:
#데이터 프레임에서 데이터를 보려면 show
df.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [30]:
df.filter('Value > 2').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [33]:
df.filter("Name == 'Bob'").show()

+----+-----+
|Name|Value|
+----+-----+
| Bob|    2|
+----+-----+



### RDBMS 데이터(테이블)

In [52]:
df.createOrReplaceTempView('tbpeople') #tbpeople이라는 테이블을 만듦

In [53]:
select_sql = 'SELECT * FROM tbpeople WHERE Value > 2'

In [55]:
spark.sql(select_sql).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



### 데이터프레임 생성 -> select로 DF를 꺼내기

In [81]:
data2 = [('최', 100), ('영',200 ), ('준',300)]
df2 = spark.createDataFrame(data2, ['Name', 'Value'])

df2.createOrReplaceTempView('test_df2')

spark.sql('select * from test_df2').show() #sql 딱히 대소문자 관계 없음


+----+-----+
|Name|Value|
+----+-----+
|  최|  100|
|  영|  200|
|  준|  300|
+----+-----+



'local[*]'

### MLlib

In [87]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [90]:
df = spark.createDataFrame([('Alice', 25), ('Bob', 30), ('Charlie', 30)], ['name', 'age'] )

df_filtered = df.filter(df.age> 28)
df_filtered.show()

+-------+---+
|   name|age|
+-------+---+
|    Bob| 30|
|Charlie| 30|
+-------+---+



In [91]:
assembler = VectorAssembler(inputCols = ['age'], outputCol = 'features')
vector_df = assembler.transform(df)

In [93]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'age')
model = lr.fit(vector_df)

24/12/03 11:29:42 WARN Instrumentation: [b90aba44] regParam is zero, which might cause numerical instability and overfitting.
24/12/03 11:29:43 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/03 11:29:43 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/03 11:29:43 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/03 11:29:43 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [95]:
pred = model.transform(vector_df)
pred.show()

+-------+---+--------+----------+
|   name|age|features|prediction|
+-------+---+--------+----------+
|  Alice| 25|  [25.0]|      25.0|
|    Bob| 30|  [30.0]|      30.0|
|Charlie| 30|  [30.0]|      30.0|
+-------+---+--------+----------+



In [96]:
#spark 종료해줌
spark.stop()

In [62]:
import seaborn as sns

In [70]:
sns.get_dataset_names()
sns_df = sns.load_dataset('anscombe')

In [80]:
sns_df.createOrReplaceTempView('sns_sample_table')
select_sql = 'select * from data'
spark.sql(select_sql).show()

#sns_df는 dataframe이라서 이 방식이 일단 안통하는 듯

AttributeError: 'DataFrame' object has no attribute 'createOrReplaceTempView'